In [1]:
!pip install music21
!pip install collections 
!pip install itertools 
!pip install grammar 
!pip install pipes
!pip install keras
!apt-get install asunder lame

    100% |████████████████████████████████| 18.0MB 2.3MB/s eta 0:00:01
  Running setup.py bdist_wheel for music21 ... done
  Stored in directory: /Users/yaron/Library/Caches/pip/wheels/53/8b/a6/be1921c60a68f0bea31c6b6a0a7b125badd61294d6a694407f
Successfully built music21
tensorflow 1.10.0 has requirement numpy<=1.14.5,>=1.13.3, but you'll have numpy 1.15.2 which is incompatible.
tensorflow 1.10.0 has requirement setuptools<=39.1.0, but you'll have setuptools 40.4.3 which is incompatible.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
  Could not find a version that satisfies the requirement collections (from versions: )
No matching distribution found for collections
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
  Could not find a version that satisfies the requirement itertools (from versions: 

# Helper functions to formats conversion

In [0]:
import os
import scipy.io.wavfile as wav
import numpy as np
from pipes import quote
#from config import nn_config

def convert_mp3_to_wav(filename, sample_frequency):
	ext = filename[-4:]
	if(ext != '.mp3'):
		return
	files = filename.split('/')
	orig_filename = files[-1][0:-4]
	orig_path = filename[0:-len(files[-1])]
	new_path = ''
	if(filename[0] == '/'):
		new_path = '/'
	for i in range(len(files)-1):
		new_path += files[i]+'/'
	tmp_path = new_path + 'tmp'
	new_path += 'wave'
	if not os.path.exists(new_path):
		os.makedirs(new_path)
	if not os.path.exists(tmp_path):
		os.makedirs(tmp_path)
	filename_tmp = tmp_path + '/' + orig_filename + '.mp3'
	new_name = new_path + '/' + orig_filename + '.wav'
	sample_freq_str = "{0:.1f}".format(float(sample_frequency)/1000.0)
	cmd = 'lame -a -m m {0} {1}'.format(quote(filename), quote(filename_tmp))
	os.system(cmd)
	cmd = 'lame --decode {0} {1} --resample {2}'.format(quote(filename_tmp), quote(new_name), sample_freq_str)
	os.system(cmd)
	return new_name

def convert_flac_to_wav(filename, sample_frequency):
	ext = filename[-5:]
	if(ext != '.flac'):
		return
	files = filename.split('/')
	orig_filename = files[-1][0:-5]
	orig_path = filename[0:-len(files[-1])]
	new_path = ''
	if(filename[0] == '/'):
		new_path = '/'
	for i in range(len(files)-1):
		new_path += files[i]+'/'
	new_path += 'wave'
	if not os.path.exists(new_path):
		os.makedirs(new_path)
	new_name = new_path + '/' + orig_filename + '.wav'
	cmd = 'sox {0} {1} channels 1 rate {2}'.format(quote(filename), quote(new_name), sample_frequency)
	os.system(cmd)
	return new_name


def convert_folder_to_wav(directory, sample_rate=44100):
	for file in os.listdir(directory):
		fullfilename = directory+file
		if file.endswith('.mp3'):
			convert_mp3_to_wav(filename=fullfilename, sample_frequency=sample_rate)
		if file.endswith('.flac'):
			convert_flac_to_wav(filename=fullfilename, sample_frequency=sample_rate)
	return directory + 'wave/'

def read_wav_as_np(filename):
	data = wav.read(filename)
	np_arr = data[1].astype('float32') / 32767.0 #Normalize 16-bit input to [-1, 1] range
	#np_arr = np.array(np_arr)
	return np_arr, data[0]

def write_np_as_wav(X, sample_rate, filename):
	Xnew = X * 32767.0
	Xnew = Xnew.astype('int16')
	wav.write(filename, sample_rate, Xnew)
	return

def convert_np_audio_to_sample_blocks(song_np, block_size):
	block_lists = []
	total_samples = song_np.shape[0]
	num_samples_so_far = 0
	print(block_size)
	while(num_samples_so_far < total_samples):
		block = song_np[num_samples_so_far:num_samples_so_far+block_size]
		if(block.shape[0] < block_size):
			padding = np.zeros((block_size - block.shape[0],))
			block = np.concatenate((block, padding))
		block_lists.append(block)
		num_samples_so_far += block_size
	return block_lists

def convert_sample_blocks_to_np_audio(blocks):
	song_np = np.concatenate(blocks)
	return song_np

def time_blocks_to_fft_blocks(blocks_time_domain):
	fft_blocks = []
	for block in blocks_time_domain:
		fft_block = np.fft.fft(block)
		new_block = np.concatenate((np.real(fft_block), np.imag(fft_block)))
		fft_blocks.append(new_block)
	return fft_blocks	

def fft_blocks_to_time_blocks(blocks_ft_domain):
	time_blocks = []
	for block in blocks_ft_domain:
		num_elems = int(block.shape[0] / 2)
		real_chunk = block[0:num_elems]
		imag_chunk = block[num_elems:]
		new_block = real_chunk + 1.0j * imag_chunk
		time_block = np.fft.ifft(new_block)
		time_blocks.append(time_block)
	return time_blocks

def convert_wav_files_to_nptensor(directory, block_size, max_seq_len, out_file, max_files=20, useTimeDomain=False):
	files = []
	for file in os.listdir(directory):
		if file.endswith('.wav'):
			files.append(directory+file)
	chunks_X = []
	chunks_Y = []
	num_files = len(files)
	if(num_files > max_files):
		num_files = max_files
	for file_idx in range(num_files):
		file = files[file_idx]
		print('Processing: ', (file_idx+1),'/',num_files)
		print('Filename: ', file)
		X, Y = load_training_example(file, block_size, useTimeDomain=useTimeDomain)
		cur_seq = 0
		total_seq = len(X)
		print(total_seq)
		print(max_seq_len)
		while cur_seq + max_seq_len < total_seq:
			chunks_X.append(X[cur_seq:cur_seq+max_seq_len])
			chunks_Y.append(Y[cur_seq:cur_seq+max_seq_len])
			cur_seq += max_seq_len
	num_examples = len(chunks_X)
	num_dims_out = block_size * 2
	if(useTimeDomain):
		num_dims_out = block_size
	out_shape = (int(num_examples), int(max_seq_len), int(num_dims_out))
	x_data = np.zeros(out_shape)
	y_data = np.zeros(out_shape)
	for n in range(num_examples):
		for i in range(max_seq_len):
			x_data[n][i] = chunks_X[n][i]
			y_data[n][i] = chunks_Y[n][i]
		print('Saved example ', (n+1), ' / ',num_examples)
	print('Flushing to disk...')
	mean_x = np.mean(np.mean(x_data, axis=0), axis=0) #Mean across num examples and num timesteps
	std_x = np.sqrt(np.mean(np.mean(np.abs(x_data-mean_x)**2, axis=0), axis=0)) # STD across num examples and num timesteps
	std_x = np.maximum(1.0e-8, std_x) #Clamp variance if too tiny
	x_data[:][:] -= mean_x #Mean 0
	x_data[:][:] /= std_x #Variance 1
	y_data[:][:] -= mean_x #Mean 0
	y_data[:][:] /= std_x #Variance 1

	np.save(out_file+'_mean', mean_x)
	np.save(out_file+'_var', std_x)
	np.save(out_file+'_x', x_data)
	np.save(out_file+'_y', y_data)
	print('Done!')

def convert_nptensor_to_wav_files(tensor, indices, filename, useTimeDomain=False):
	num_seqs = tensor.shape[1]
	for i in indices:
		chunks = []
		for x in range(num_seqs):
			chunks.append(tensor[i][x])
		save_generated_example(filename+str(i)+'.wav', chunks,useTimeDomain=useTimeDomain)

def load_training_example(filename, block_size=2048, useTimeDomain=False):
	data, bitrate = read_wav_as_np(filename)
	x_t = convert_np_audio_to_sample_blocks(data, block_size)
	y_t = x_t[1:]
	y_t.append(np.zeros(block_size)) #Add special end block composed of all zeros
	if useTimeDomain:
		return x_t, y_t
	X = time_blocks_to_fft_blocks(x_t)
	Y = time_blocks_to_fft_blocks(y_t)
	return X, Y

def save_generated_example(filename, generated_sequence, useTimeDomain=False, sample_frequency=44100):
	if useTimeDomain:
		time_blocks = generated_sequence
	else:
		time_blocks = fft_blocks_to_time_blocks(generated_sequence)
	song = convert_sample_blocks_to_np_audio(time_blocks)
	write_np_as_wav(song, sample_frequency, filename)
	return

def audio_unit_test(filename, filename2):
	data, bitrate = read_wav_as_np(filename)
	time_blocks = convert_np_audio_to_sample_blocks(data, 1024)
	ft_blocks = time_blocks_to_fft_blocks(time_blocks)
	time_blocks = fft_blocks_to_time_blocks(ft_blocks)
	song = convert_sample_blocks_to_np_audio(time_blocks)
	write_np_as_wav(song, bitrate, filename2)
	return

# Upload MP3 to directory

In [5]:
!mkdir mymodeldir
!mkdir mysongdir
!mkdir mysongdirwave
import os
os.chdir("mysongdir")

from google.colab import files
files.upload()

os.chdir("../")

mkdir: cannot create directory ‘mymodeldir’: File exists
mkdir: cannot create directory ‘mysongdir’: File exists
mkdir: cannot create directory ‘mysongdirwave’: File exists


Saving mysong.mp3 to mysong.mp3


# Weights upload to directory

In [0]:
!mkdir myweights
import os
os.chdir("myweights")

from google.colab import files
#files.upload()

os.chdir("../")

In [7]:
!ls mysongdir
!ls myweights


mysong.mp3


# Seed and Sequence Generator


In [0]:
#A very simple seed generator
#Copies a random example's first seed_length sequences as input to the generation algorithm
def generate_copy_seed_sequence(seed_length, training_data):
	num_examples = training_data.shape[0]
	randIdx = np.random.randint(num_examples, size=1)[0]
	randSeed = np.concatenate(tuple([training_data[randIdx + i] for i in range(seed_length)]), axis=0)
	seedSeq = np.reshape(randSeed, (1, randSeed.shape[0], randSeed.shape[1]))
	return seedSeq

In [0]:

#Extrapolates from a given seed sequence
def generate_from_seed(model, seed, sequence_length, data_variance, data_mean):
	seedSeq = seed.copy()
	output = []

	#The generation algorithm is simple:
	#Step 1 - Given A = [X_0, X_1, ... X_n], generate X_n + 1
	#Step 2 - Concatenate X_n + 1 onto A
	#Step 3 - Repeat MAX_SEQ_LEN times

	print (seedSeq.shape)

	result = np.zeros( (1, seedSeq.shape[1]+0, seedSeq.shape[2] ) )
	result[ 0, range(0,seedSeq.shape[1]+0), :] = seedSeq
	seedSeqNew = model.predict(result) #Step 1. Generate X_n + 1
		

	for it in range(sequence_length):
		print ('it: ', it )
		print (seedSeq.shape)
		seedSeqNew = model.predict(seedSeq) #Step 1. Generate X_n + 1
		#Step 2. Append it to the sequence
		if it == 0:
			for i in range(seedSeqNew.shape[1]):
				output.append(seedSeqNew[0][i].copy())
		else:
			output.append(seedSeqNew[0][seedSeqNew.shape[1]-1].copy()) 
		newSeq = seedSeqNew[0][seedSeqNew.shape[1]-1]
		newSeq = np.reshape(newSeq, (1, 1, newSeq.shape[0]))
		print (newSeq.shape)
		seedSeq = np.concatenate((seedSeq, newSeq), axis=1)

		result = np.zeros( (1, seedSeq.shape[1]-1, seedSeq.shape[2] ) )
		result[ 0, range(0,seedSeq.shape[1]-1), :] = seedSeq[ 0, range(1,seedSeq.shape[1]), : ]
		seedSeq = result



	#Finally, post-process the generated sequence so that we have valid frequencies
	#We're essentially just undo-ing the data centering process
	for i in range(len(output)):
		output[i] *= data_variance
		output[i] += data_mean
	return output

# Network parameters


In [0]:
def get_neural_net_configuration():
	nn_params = {}
	nn_params['sampling_frequency'] = 44100
	#Number of hidden dimensions.
	#For best results, this should be >= freq_space_dims, but most consumer GPUs can't handle large sizes
	nn_params['hidden_dimension_size'] = 1024
	#The weights filename for saving/loading trained models
	nn_params['model_basename'] = './myweights/'
	#The model filename for the training data
	nn_params['model_file'] = './mymodeldir/'
	#The dataset directory
	nn_params['dataset_directory'] = './mysongdir/'
	return nn_params


In [11]:
!chmod 777 mysongdir/tmp/*

chmod: cannot access 'mysongdir/tmp/*': No such file or directory


# Model creation function

In [12]:
from keras.models import Sequential
from keras.layers import TimeDistributed, Dense
from keras.layers.recurrent import LSTM, GRU
import keras.layers.wrappers
from keras.layers.wrappers import TimeDistributed
from keras.layers import Dense

def create_lstm_network(num_frequency_dimensions, num_hidden_dimensions, sz, num_recurrent_units=1):
	model = Sequential()
	#This layer converts frequency space to hidden space
	model.add( TimeDistributed( Dense( num_hidden_dimensions ), input_shape=sz ) )
	#model.add(TimeDistributed(Dense(num_hidden_dimensions),input_shape=(40, num_frequency_dimensions)))
	for cur_unit in range(num_recurrent_units):
		model.add(LSTM(input_dim=num_hidden_dimensions, output_dim=num_hidden_dimensions, return_sequences=True))
	#This layer converts hidden space back to frequency space
	#model.add(TimeDistributedDense(input_dim=num_hidden_dimensions, output_dim=num_frequency_dimensions))
	model.add(TimeDistributed(Dense(num_frequency_dimensions),input_shape=(40, num_frequency_dimensions)))
	model.compile(loss='mean_squared_error', optimizer='rmsprop')
	return model


Using TensorFlow backend.


# Parse file

In [13]:
#from data_utils.parse_files import *
#import config.nn_config as nn_config

config = get_neural_net_configuration()
input_directory = config['dataset_directory']
output_filename = config['model_file'] 

sample_frequency = config['sampling_frequency'] #sample frequency in Hz
clip_len = 10 		#length of clips for training. Defined in seconds
block_size = sample_frequency / 4 #block sizes used for training - this defines the size of our input state
max_seq_len = int(round((sample_frequency * clip_len) / block_size)) #Used later for zero-padding song sequences
#Step 1 - convert MP3s to WAVs
new_directory = convert_folder_to_wav(input_directory, sample_frequency)
#Step 2 - convert WAVs to frequency domain with mean 0 and standard deviation of 1
convert_wav_files_to_nptensor(new_directory, int(block_size), max_seq_len, output_filename)

Processing:  1 / 1
Filename:  ./mysongdir/wave/mysong.wav
11025
848
40
Saved example  1  /  21
Saved example  2  /  21
Saved example  3  /  21
Saved example  4  /  21
Saved example  5  /  21
Saved example  6  /  21
Saved example  7  /  21
Saved example  8  /  21
Saved example  9  /  21
Saved example  10  /  21
Saved example  11  /  21
Saved example  12  /  21
Saved example  13  /  21
Saved example  14  /  21
Saved example  15  /  21
Saved example  16  /  21
Saved example  17  /  21
Saved example  18  /  21
Saved example  19  /  21
Saved example  20  /  21
Saved example  21  /  21
Flushing to disk...
Done!


In [14]:
!ls mysongdir/wave

mysong.wav


# Train

In [24]:
from __future__ import absolute_import
from __future__ import print_function
import numpy as np
import os

config = get_neural_net_configuration()
inputFile = config['model_file']
cur_iter = 0
model_basename = config['model_basename']
model_filename = model_basename + str(cur_iter)

#Load up the training data
print ('Loading training data')
#X_train is a tensor of size (num_train_examples, num_timesteps, num_frequency_dims)
#y_train is a tensor of size (num_train_examples, num_timesteps, num_frequency_dims)
X_train = np.load(inputFile + '_x.npy')
y_train = np.load(inputFile + '_y.npy')
print ('Finished loading training data')

#Figure out how many frequencies we have in the data
freq_space_dims = X_train.shape[2]
hidden_dims = config['hidden_dimension_size']

print (X_train.shape)
print (hidden_dims)
#Creates a lstm network
model = create_lstm_network(num_frequency_dimensions=freq_space_dims, num_hidden_dimensions=hidden_dims, sz=( X_train.shape[1], X_train.shape[2]) )
#You could also substitute this with a RNN or GRU
#model = network_utils.create_gru_network()

#Load existing weights if available
if os.path.isfile(model_filename):
	model.load_weights(model_filename)

num_iters = 1000 			#Number of iterations for training
epochs_per_iter = 60	#Number of iterations before we save our model
batch_size = 90			#Number of training examples pushed to the GPU per batch.
						#Larger batch sizes require more memory, but training will be faster
print ('Starting training!')
while cur_iter < num_iters:
	print('Iteration: ' + str(cur_iter))
	#We set cross-validation to 0,
	#as cross-validation will be on different datasets 
	#if we reload our model between runs
	#The moral way to handle this is to manually split 
	#your data into two sets and run cross-validation after 
	#you've trained the model for some number of epochs
	history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs_per_iter, verbose=1, validation_split=0.0)
	cur_iter += epochs_per_iter
	model.save_weights(model_basename + str(cur_iter))
print ('Training complete!')

Loading training data
Finished loading training data
(21, 40, 22050)
1024


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(return_sequences=True, input_shape=(None, 102..., units=1024)`
  


Starting training!
Iteration: 0
Epoch 1/60
21/21 [==============================] - 18s 877ms/step - loss: 1.0027
Epoch 2/60
21/21 [==============================] - 13s 605ms/step - loss: 1.0400
Epoch 3/60
21/21 [==============================] - 13s 596ms/step - loss: 0.9845
Epoch 4/60


----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 47018, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 696, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil

21/21 [==============================] - 13s 600ms/step - loss: 1.0369
Epoch 5/60
21/21 [==============================] - 12s 578ms/step - loss: 0.9791
Epoch 6/60
21/21 [==============================] - 13s 599ms/step - loss: 0.9183
Epoch 7/60
21/21 [==============================] - 12s 588ms/step - loss: 0.9172
Epoch 8/60
21/21 [==============================] - 13s 602ms/step - loss: 0.8932
Epoch 9/60
21/21 [==============================] - 13s 604ms/step - loss: 0.8890
Epoch 10/60
21/21 [==============================] - 12s 573ms/step - loss: 0.8828
Epoch 11/60
21/21 [==============================] - 12s 583ms/step - loss: 0.9048
Epoch 12/60
21/21 [==============================] - 12s 581ms/step - loss: 0.8705
Epoch 13/60
21/21 [==============================] - 12s 577ms/step - loss: 0.9228
Epoch 14/60
21/21 [==============================] - 13s 596ms/step - loss: 0.8615
Epoch 15/60
21/21 [==============================] - 12s 576ms/step - loss: 0.9061
Epoch 16/60
21/21 [==

In [26]:
try:
	files.download(model_basename + str(cur_iter))
	print ('Downloaded weights %s!'%(model_basename + str(cur_iter)))
except:
	print ('Couldnt downloaded weights')

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 34418, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 696, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil

Downloaded weights ./myweights/1020!


In [30]:
cur_iter = 1020
!ls myweights

1020  180  300	420  540  600  720  840  960
120   240  360	480  60   660  780  900


# Generate

In [31]:
from __future__ import absolute_import
from __future__ import print_function

config = get_neural_net_configuration()
sample_frequency = config['sampling_frequency']
inputFile = config['model_file']
model_basename = config['model_basename']
model_filename = model_basename + str(cur_iter)
output_filename = './generated_song_iter_' + str(cur_iter) + '.wav'

#Load up the training data
print ('Loading training data')
#X_train is a tensor of size (num_train_examples, num_timesteps, num_frequency_dims)
#y_train is a tensor of size (num_train_examples, num_timesteps, num_frequency_dims)
#X_mean is a matrix of size (num_frequency_dims,) containing the mean for each frequency dimension
#X_var is a matrix of size (num_frequency_dims,) containing the variance for each frequency dimension
X_train = np.load(inputFile + '_x.npy')
y_train = np.load(inputFile + '_y.npy')
X_mean = np.load(inputFile + '_mean.npy')
X_var = np.load(inputFile + '_var.npy')
print ('Finished loading training data')

#Figure out how many frequencies we have in the data
freq_space_dims = X_train.shape[2]
hidden_dims = config['hidden_dimension_size']
print (X_train.shape)

#Creates a lstm network
model = create_lstm_network(num_frequency_dimensions=freq_space_dims, num_hidden_dimensions=hidden_dims, sz=( X_train.shape[1], X_train.shape[2] ) )
#You could also substitute this with a RNN or GRU
#model = network_utils.create_gru_network()

#Load existing weights if available
if os.path.isfile(model_filename):
	model.load_weights(model_filename)
else:
	print('Model filename ' + model_filename + ' could not be found!')

print ('Starting generation!')
#Here's the interesting part
#We need to create some seed sequence for the algorithm to start with
#Currently, we just grab an existing seed sequence from our training data and use that
#However, this will generally produce verbatum copies of the original songs
#In a sense, choosing good seed sequences = how you get interesting compositions
#There are many, many ways we can pick these seed sequences such as taking linear combinations of certain songs
#We could even provide a uniformly random sequence, but that is highly unlikely to produce good results
seed_len = 1
seed_seq = generate_copy_seed_sequence(seed_length=seed_len, training_data=X_train)

print ('seed seq:')
print (seed_seq.shape)

max_seq_len = 10; #Defines how long the final song is. Total song length in samples = max_seq_len * example_len
output = generate_from_seed(model=model, seed=seed_seq, sequence_length=max_seq_len, data_variance=X_var, data_mean=X_mean)
print ('Finished generation!')

#Save the generated sequence to a WAV file
save_generated_example(output_filename, output, sample_frequency=sample_frequency)

Loading training data
Finished loading training data
(21, 40, 22050)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(return_sequences=True, input_shape=(None, 102..., units=1024)`
  


Starting generation!
seed seq:
(1, 40, 22050)
(1, 40, 22050)
it:  0
(1, 40, 22050)
(1, 1, 22050)
it:  1
(1, 40, 22050)
(1, 1, 22050)
it:  2
(1, 40, 22050)
(1, 1, 22050)
it:  3
(1, 40, 22050)
(1, 1, 22050)
it:  4
(1, 40, 22050)
(1, 1, 22050)
it:  5
(1, 40, 22050)
(1, 1, 22050)
it:  6
(1, 40, 22050)
(1, 1, 22050)
it:  7
(1, 40, 22050)
(1, 1, 22050)
it:  8
(1, 40, 22050)
(1, 1, 22050)
it:  9
(1, 40, 22050)
(1, 1, 22050)
Finished generation!


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:72: ComplexWarning: Casting complex values to real discards the imaginary part


# Download song

In [0]:
files.download(output_filename) 